In [1]:
import numpy as np
import csv
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
#from tensorflow.examples.tutorials.mnist import input_data
from torch.autograd import Variable
import sklearn.metrics as metrics
import pickle

In [2]:
with open('screen_info.txt','rb') as fl:
    t = pickle.load(fl)
fnames = t[0]
totf = t[1]
binf = t[2]
runfile = 1
fname = fnames[runfile]
bf = binf[runfile]

In [3]:
path = os.getcwd() + '/bioassay-datasets/'
p_fingerprints = []
c_fingerprints = []
labels = []
with open(path+fname+'red_train.csv') as csvfile:
    readcsv = csv.reader(csvfile)
    for row in readcsv:
        p_fingerprints.append(row[:bf])
        c_fingerprints.append(row[bf:-1])
        labels.append(row[-1])

###  Using 10 fold cross validation 

In [4]:
print(np.asarray(p_fingerprints)[1:].shape)

(47831, 122)


In [5]:
p_fingerprints = np.asarray(p_fingerprints)[1:]
p_fingerprints = p_fingerprints.astype(int)
c_fingerprints = np.asarray(c_fingerprints)[1:]
c_fingerprints = c_fingerprints.astype(float)
#p2_fingerprints = np.ones(p_fingerprints.shape)
(no_examples , ip_dim) = p_fingerprints.shape
ip_dim2 = c_fingerprints.shape[1]
labels = labels[1:]
print(no_examples,ip_dim)
print(ip_dim2)
#p_fingerprints[(p_fingerprints==0)] = -1

(47831, 122)
32


In [6]:
labels2 = np.zeros((len(labels),1))
for i,l in enumerate(labels):
    if l=='Active':
        labels2[i] = 1
    else:
        labels2[i] = 0
labels2 = labels2.astype(int)

#####  Randomly permute the data

In [7]:
ind = np.arange(no_examples)
np.random.shuffle(ind)
p_fingerprints = p_fingerprints[ind]
c_fingerprints = c_fingerprints[ind]
labels2 = labels2[ind]

In [8]:
no_active_ele = (sum(labels2))
print(no_active_ele)

[170]


In [21]:
class mlp(nn.Module):
    def __init__(self):
        super(mlp,self).__init__()
        self.l1 = nn.Linear(ip_dim,1000)
        self.l2 = nn.Linear(1000,500)
        self.l3 = nn.Linear(500,500)
        self.l4 = nn.Linear(500,200)
        self.l5 = nn.Linear(200,2)
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = F.leaky_relu(self.l3(x))
        x = F.leaky_relu(self.l4(x))
        x = (self.l5(x))
        
        return x

In [22]:

#criterion = nn.BCELoss()

batch_size = 128

In [23]:
def get_train_batch(batch_size,validation_iter=0,binary=True):
    
    if validation_iter == 0: #no validation
        curr_data_size = no_examples
        labels_train = labels2
    else:
        curr_data_size = int(no_examples*0.8)
        interval_size = int(no_examples*0.2)
        
        if(val_iter==1):
            s_ind1 = int((validation_iter)*interval_size)
            end_ind1 = int((validation_iter+1)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        else:
            s_ind1 = 0
            end_ind1 = int((validation_iter)*interval_size)
            s_ind2 = int((validation_iter + 1) * interval_size)
            end_ind2 = int(no_examples)
        
        indices = range(s_ind1,end_ind1) + range(s_ind2,end_ind2)
        c_train_data = c_fingerprints[indices]
        p_train_data = p_fingerprints[indices]
        labels_train = labels2[indices]
                               
    samples = np.random.randint(low=0,high=curr_data_size,size=(batch_size,1))
    if binary == True:
        train_batch = p_fingerprints[samples].reshape(batch_size,ip_dim)
        train_batch = train_batch.astype(int)
    else:
        train_batch = c_fingerprints[samples].reshape(batch_size,ip_dim2)
        train_batch = train_batch.astype(float)
    
    train_batch = torch.cuda.FloatTensor(train_batch)
    train_batch = Variable(train_batch,requires_grad=False).cuda()
    target = Variable(torch.cuda.LongTensor(labels_train[samples]),requires_grad=False)
    target = target.view(batch_size,)
    return train_batch,target

In [24]:
def get_val_data(validation_iter):
    interval_size = int(no_examples)*0.2
    s_ind = int((validation_iter-1)*interval_size)
    e_ind = int((validation_iter) * interval_size)
    p_train_data = p_fingerprints[s_ind:e_ind]
    labels_val = labels2[s_ind:e_ind]   
    return Variable(torch.cuda.FloatTensor(p_train_data)),labels_val

In [25]:
# with open("good_w_binary.txt",'rb') as f:
#     weights_array = pickle.load(f)

In [26]:
min_fn = 15
max_fp = 60
cm_list = []
get_model  = 1
for val_iter in range(1,6):
    print("val iter: ",val_iter)
    
   
    #weights_array = [5]
    weights_array = np.linspace(5,25,20)
    for i,w in enumerate(weights_array): 
        mymlp = mlp().cuda()
        optimizer = torch.optim.Adagrad(mymlp.parameters(),lr=1e-3)
        criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor([1,w]))

        for ep in range(3000):
            train_batch,target = get_train_batch(batch_size,validation_iter = val_iter)
            model_op = mymlp(train_batch)
            #print(model_op.type)
            #print(target.type)
            loss = criterion(model_op,target)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    #     if(ep%30==29):
    #         print(loss.data[0])

        ## After training check on cross validation data
        val_data,labels_val = get_val_data(val_iter)
        train_op = mymlp(val_data)
        train_op = train_op.cpu().data.numpy()
        pred_labels = np.argmax(train_op,axis=1)
        #tmp_labels = tmp_labels.data.cpu().numpy()
        #print(sum(tmp_labels))
        cf = metrics.confusion_matrix(labels_val,pred_labels).ravel()
        #print(val_iter,w)
        print('tn, fp, fn, tp: ',cf)
        [tn,fp,fn,tp] = cf
        wcf = [val_iter] + [w] + [cf]
        if(fn < 10):
            cm_list.append(wcf)
            if(fn==0):
                if(fp <= max_fp+2):
                    max_fp = fp
                    model_path = os.getcwd()+'/b_model_lrelu_'+fname + '_' + str(get_model)
                    torch.save(mymlp.state_dict(),model_path)
                    print("saving model on val: ",val_iter," and weight: ",w)
                    if(get_model<4):
                        get_model = get_model + 1
                    else:
                        break

        


('val iter: ', 1)
('tn, fp, fn, tp: ', array([9525,    0,   41,    0]))
('tn, fp, fn, tp: ', array([9524,    1,   41,    0]))
('tn, fp, fn, tp: ', array([9525,    0,   41,    0]))
('tn, fp, fn, tp: ', array([9525,    0,   41,    0]))
('tn, fp, fn, tp: ', array([9524,    1,   41,    0]))
('tn, fp, fn, tp: ', array([9524,    1,   41,    0]))
('tn, fp, fn, tp: ', array([9524,    1,   41,    0]))
('tn, fp, fn, tp: ', array([9524,    1,   41,    0]))
('tn, fp, fn, tp: ', array([9525,    0,   41,    0]))
('tn, fp, fn, tp: ', array([9523,    2,   41,    0]))
('tn, fp, fn, tp: ', array([9523,    2,   41,    0]))
('tn, fp, fn, tp: ', array([9521,    4,   41,    0]))
('tn, fp, fn, tp: ', array([9503,   22,   41,    0]))
('tn, fp, fn, tp: ', array([9516,    9,   41,    0]))
('tn, fp, fn, tp: ', array([9525,    0,   41,    0]))
('tn, fp, fn, tp: ', array([9503,   22,   41,    0]))
('tn, fp, fn, tp: ', array([9510,   15,   41,    0]))
('tn, fp, fn, tp: ', array([9512,   13,   41,    0]))
('tn, fp, 

In [ ]:
cm_lsit

In [ ]:
# with open("selected_c_cm.txt",'wb') as fp:
#     pickle.dump(cm_lsit,fp)

In [ ]:
# import pickle
# with open("test3.txt",'rb') as fp:
#     tmp_list = pickle.load(fp)

In [ ]:
n_eg = 5

In [ ]:
def tmpfun(y):
    z = n_eg*3
    print(z)

In [ ]:
x = 20
tmpfun(3)
print(n_eg)